In [1]:
import pandas as pd
from numpy import genfromtxt
from itertools import chain
import sys
from collections import Counter
import functools

In [2]:
import os
os.chdir('../')
os.getcwd()

'/home/redgar/Documents/organoid_passage_DNAm'

I have exported the annotation of the EPIC from R in the previous script. Here I use hg18/build36 as the cDMR annotation has genome build 36 coordinates.

In [3]:
# https://www.nature.com/articles/ng.298#Sec25
# supplementary data 2
Feature_bed = 'data/41588_2009_BFng298_MOESM18_ESM.csv'
background = 'data/passage_background_build36.csv'

Feature_bed = pd.read_csv(Feature_bed, header=1,skiprows=2)
CpG_background_pd = pd.read_csv(background)

Feature_bed['chr'] = Feature_bed['chr'].str.replace('chr','')


Pulling a range of CpGs for an example of the annotation. The actual code takes the reange as arguments as commented out below.

In [6]:
CpG_start = 15690
CpG_end = 15698

#CpG_start = int(sys.argv[1])
#CpG_end = int(sys.argv[1])+10000

This pulls on cDMR on the same chromosome as a CpG, then if a CpG is within a cDMR it returns the delta M of that cDMR so we can later compared direction of DNAm change. 

In [7]:
# subset to system arguments
CpG_background_pd = CpG_background_pd[CpG_start:CpG_end]

### Actual background SNPs in TFBS
EPIC_features=[]

for snp, row in CpG_background_pd.iterrows():
    # match chromosome
    chr_features = Feature_bed.loc[Feature_bed['chr'] == str(row['Chromosome_36'])]

    features_snp_in = []
    for feat in range(0,len(chr_features.index)):#chr_TFBS.size
        if chr_features['start'].iloc[feat] <= int(row['Coordinate_36']) <= chr_features['end'].iloc[feat]:
            features_snp_in.append(chr_features['delta M'].iloc[feat])

    EPIC_features.append([row['IlmnID'], features_snp_in])

df = pd.DataFrame(EPIC_features)

In [10]:
chr_features = Feature_bed.loc[Feature_bed['chr'] == str(19)]
chr_features.head(4)

,Unnamed: 0,chr,start,end,delta M,colon.tumor,colon.normal,fdr,state,name,annotation,region,distance,island,distToIsla
5,31523,19,5159373,5160095,-1.014437,1.521337,2.518715,0.0,Less methylation,PTPRS,NM_130854,inside,131718,Far,1469
6,31686,19,5297115,5297780,-0.932921,0.299673,1.256909,0.0,No methylation,PTPRS,NM_130854,upstream,5302,Far,-5054
7,32005,19,55631525,55632172,-0.846214,0.704149,1.674941,0.0,Less methylation,MYBPC2,NM_004533,inside,29216,Shore,-4036
8,32467,19,60702435,60703541,-0.930610,1.307662,2.173489,0.0,Less methylation,NAT14,NM_020378,upstream,11689,Shore,3082


In [11]:
df

,0,1
0,cg10994086,[]
1,cg25613541,[]
2,cg20741185,[]
3,cg23474190,[-1.163675835]
4,cg24882192,[]
5,cg07668358,[]
6,cg11285206,[]
7,cg14298379,[]


In [ ]:
df.to_csv("data/annotation_split/EPIC_Features_cDMR36_" + sys.argv[1] + ".csv") 